In [80]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key

In [81]:
#read csv file
city_data = pd.read_csv("Output_data/cities.csv" ,index_col=0)
city_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,albany,6,US,1666412740,72,42.6001,-73.9662,46.81,3.31
1,yar-sale,100,RU,1666412740,98,66.8333,70.8333,30.04,3.47
2,punta arenas,98,CL,1666412601,69,-53.1500,-70.9167,46.11,28.05
3,sitka,100,US,1666412741,87,57.0531,-135.3300,42.69,5.75
4,vardo,75,NO,1666412741,93,70.3705,31.1107,33.78,23.02


# Humidity Heatmap

In [82]:
#configure gmaps with API key
gmaps.configure(api_key=g_key)


In [83]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_data[["Lat", "Lng"]].astype(float)
weights= city_data["Humidity"]

# Create HeatMap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations,weights = weights,
                              dissipating=False, max_intensity=100,
                              point_radius = 3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [84]:
# A max temperature lower than 80 degrees but higher than 70
# Wind speed less than 10 mph
# Zero cloudiness
new_df = city_data.loc[(city_data["Max Temp"]>=70) & (city_data["Max Temp"]<80)]
new_df = new_df.loc[(new_df["Wind Speed"]>10) & (new_df["Cloudiness"]==0)]
filtered_df = new_df.reset_index(drop=True)
#remove rows with null values
filtered_df.dropna()
#ctreating new dataframe with few columns and new column
hotel_df = filtered_df[["City","Country","Cloudiness","Lat","Lng","Max Temp","Wind Speed"]]
hotel_df["Hotel Name"] = ""
hotel_df

C:\Users\13135\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,City,Country,Cloudiness,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,atuona,PF,0,-9.8000,-139.0333,76.98,14.32,
1,carnarvon,AU,0,-24.8667,113.6333,73.67,14.38,
2,jasper,US,0,37.2001,-94.3502,73.00,14.97,
3,gatesville,US,0,31.4352,-97.7439,72.36,16.42,
4,arlington,US,0,32.7357,-97.1081,77.32,18.41,


In [85]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key}

#looping through the dataframe
for index,row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat},{lng}"
    
     # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
     # convert to json
    response = hotel_name.json()
    try:
        hotel_df.loc[index,"Hotel Name"]=response['results'][0]['name']        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
#     pprint(response)

C:\Users\13135\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [86]:
# display the dataframe with new values
hotel_df

,City,Country,Cloudiness,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,atuona,PF,0,-9.8000,-139.0333,76.98,14.32,Hôtel Hiva Oa Hanakee Lodge
1,carnarvon,AU,0,-24.8667,113.6333,73.67,14.38,Carnarvon Hotel
2,jasper,US,0,37.2001,-94.3502,73.00,14.97,Days Inn by Wyndham Carthage
3,gatesville,US,0,31.4352,-97.7439,72.36,16.42,Holiday Inn Express & Suites Gatesville - N. F...
4,arlington,US,0,32.7357,-97.1081,77.32,18.41,Courtyard by Marriott Dallas Arlington/Enterta...


In [78]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]].astype(float)

# Create HeatMap

fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig



Figure(layout=FigureLayout(height='420px'))

In [79]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))